In [15]:
# install some of the modules first
pip install mouse
pip install keyboard
pip install pickle

SyntaxError: invalid syntax (<ipython-input-15-6958f4e8420c>, line 2)

In [2]:
import mouse
import keyboard
import pickle

file = input('Mouse movement name?? :\n')
file = file + '.txt'

# simple recording of mouse events

# put to list
mouse_events = []

# start recording
mouse.hook(mouse_events.append)


#kb = keyboard.record(until ='Esc')

#press Esc to kill the event
if not keyboard.wait("Esc"):
    mouse.unhook_all()
    # save the recording    
    with open(file, 'wb') as fh:
       pickle.dump(mouse_events, fh)


Mouse movement name?? :
mouse1


In [ ]:
# open the recording
pickle_off = open (file, "rb")
emp = pickle.load(pickle_off)


# replay the events
mouse.play(emp)

# Enhanced version to combine mouse and keyboard

In [19]:

import mouse
import keyboard            
import pickle
import pandas as pd



file = input('Step Name ? :\n')
file = file + '.txt'



mouse_events = []


mouse.hook(mouse_events.append)
keyboard.start_recording()

keyboard.wait("Esc")

mouse.unhook(mouse_events.append)
keyboard_events = keyboard.stop_recording()


# put mouse results (click only) to list
list = []

for i in range(len(mouse_events)):
    if mouse_events[i][0] == 'down':
        # pick up previus coordinate
        #row = i - 1
        #x = mouse_events[row][0]
        #y = mouse_events[row][1]     
        #list.append([x,y])
        
        for chk in range(1,5):
        #    # check previous 5 rows until you dont see 'down' or 'up' key
            row = i - chk
            if mouse_events[row][0] not in ('down','up'):
                 x = mouse_events[row][0]
                 y = mouse_events[row][1]     
                 z = mouse_events[row][2]
                 list.append(['Mouse Click Event',x,y,z])
                 break   
                
# put keyboard results to list
string = []
stroke = []
finalword = []

for m in range(len(keyboard_events)):
    mystring = str(keyboard_events[m])
    letter = mystring[14:15]
    if 'down' in mystring:
        if 'backspace' in mystring:
            #print (string)
            string.pop()
            #print (string)
            #list.append(['Keyboard Event','backspace'])
        elif 'space' in mystring:
             #list.append(['Keyboard Event','space'])
             string.append(' ')
        elif 'enter' in mystring:
               
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','enter'])
                string = []
            
            #finalword = []
            #string.append(' Enter ')
        elif 'tab' in mystring:
            #list.append(['Keyboard Event','enter'])
            #string.append(' Tab ')
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','tab'])
                string = []
            
        elif 'shift' in mystring:
            # shift means nothing for this logic as it is just uppercase or lowercase
            letter = ''
            
        elif 'esc' in mystring:
            #list.append(['Ignore','esc'])
            #print (mystring)
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','enter'])
                string = []
        else:
            string.append(letter)
            
            
# add duration to the list

for index, i in enumerate(list):
   try :
    if index != 0:
        now = list[index][3]
        before = list[index-1][3]
        duration = now - before
        list[index].append(duration)
    else:
        list[index].append(0)
   except:
        break
        
        
# tidy up the sequence between mouse and keyboards into a list

import pyautogui as s
import time

tmp = []
newlist = []
#count = 0

for index, i in enumerate(list):
    if 'Mouse' in i[0]:             # mouse in string 
        time = i[4]                 # time column
        if time < 2 and tmp == []:
            newlist.append(i)
        else:
            # if time > 2 seconds, move them into tmp list
            tmp.append(i)
    elif 'Keyboard Write' in i[0]:
        newlist.append(i)
    elif 'Keyboard Event' in i[0]:
        newlist.append(i)
        
        
        # first check if it is a tab event
        if i[0][1] == 'tab':
            # add the next two events to the list
            newlist.append(i+1)
            newlist.append(i+2)
        
       
        
        else:
        # after keyboard events will bring back those mouse events
        # from tmp list after the keyboards.
        # logic will loop until the next mouse event > 2 seconds
        # at the moment assumption is typing will be more than 2 seconds.
            for i in range(len(tmp)):

                if i == 0:     # first event, straight back to the main list
                    count = 0
                    newlist.append(tmp[count])
                    count += 1
                    #tmp.pop(i)
                    #i = i - 1
                elif tmp[i][4] < 2:      # if less than 2 seconds, keep adding to main list
                    newlist.append(tmp[count])
                    count += 1
                    #tmp.pop(i)
                    #i = i -1
                elif tmp[i][4] > 2:      # when the next keyboard even found (> 2 seconds), exit the loop
                    break
        
# if no keyboard events, length of new list will be shorter than the original list
# hence just use the original list

if len(newlist) != len(list):
    newlist = list
            
# save the recording    
with open('list.txt', 'wb') as fh:
   pickle.dump(list, fh)

# save the recording    
with open('newlist.txt', 'wb') as fh:
   pickle.dump(newlist, fh)

# save the recording    
with open('mouse.txt', 'wb') as fh:
   pickle.dump(mouse_events, fh)

# save the recording    
with open('keyboard.txt', 'wb') as fh:
   pickle.dump(keyboard_events, fh)


# put to dataframes
x1 = pd.DataFrame(list)
x2 = pd.DataFrame(newlist)
x1.columns = ['Event','X','Y','Time','Duration']
x2.columns = ['Event','X','Y','Time','Duration']
print ('Initial List \n',x1,'\n\n','New list \n',x2)

   

Initial List 
                Event          X      Y          Time  Duration
0  Mouse Click Event        151  743.0  1.637408e+09       0.0
1     Keyboard Write    command    NaN           NaN       NaN
2     Keyboard Event      enter    NaN           NaN       NaN
3     Keyboard Write  cd videos    NaN           NaN       NaN
4     Keyboard Event      enter    NaN           NaN       NaN 

 New list 
                Event          X      Y          Time  Duration
0  Mouse Click Event        151  743.0  1.637408e+09       0.0
1     Keyboard Write    command    NaN           NaN       NaN
2     Keyboard Event      enter    NaN           NaN       NaN
3     Keyboard Write  cd videos    NaN           NaN       NaN
4     Keyboard Event      enter    NaN           NaN       NaN


In [20]:
# replay the click        
import pyautogui as s
import time

#newlist = list

for i in range(len(newlist)):
    if 'Mouse' in newlist[i][0]:
        x = newlist[i][1]
        y = newlist[i][2]
        s.moveTo(x,y,0.7)
        s.click()
        time.sleep(2.5)
    elif 'Write' in newlist[i][0]:
        #break
        #s.press('enter')
        s.write(newlist[i][1],0.3)
    elif 'tab' in newlist[i][1]:
        s.press('tab')
        time.sleep(0.8)
    elif 'enter' in newlist[i][1]:
        s.press('enter')
        time.sleep(2.5)

# Once recorded, you can add parameters to the list

1. Define the parameter in param
2. specific the location of the index tochange variable
3. store the new variable under x
4. put that x value to the list

Re-run the replay

In [ ]:
# open the text file where you recorded previous events
pickle_off = open ('SAP1.txt', "rb")
modified = pickle.load(pickle_off)


In [11]:
# to remove the last event, if you want to remove specific index type the number inside the bracket
modified.pop()

# inserting to a specific index of additional code
modified[7] = (['Keyboard Event','enter'])

# adding to the last row
modified.append(['Keyboard Event','enter'])

# mergin one list to another, e.g. previous file called old
modified.append(old[0])

# adding parameters
param = '2020-06-30'
tochange = modified[3][1]

x = str(tochange) + ' ' + str(param)
modified[3][1] = x

print (x)


['Mouse Click Event', 788, 20, 1637407689.5779638, 0]

In [62]:
# after modified, save the text file again   
with open('SAP1.txt', 'wb') as fh:
   pickle.dump(modified, fh)
